# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
cityDataDF = pd.read_csv("../output_data/cities.csv")

# Display sample data
cityDataDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,susuman,62.7833,148.1667,7.46,73,2,1.25,RU,1691690000
1,1,alo,7.4178,6.7371,25.84,90,97,1.51,NG,1691645201
2,2,ushuaia,-54.8000,-68.3000,4.81,81,75,4.12,AR,1691669126
3,3,saint-leu,-21.1500,55.2833,20.48,91,16,1.21,RE,1691635537
4,4,invercargill,-46.4000,168.3500,1.12,91,100,1.79,NZ,1691697271


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot_1 = cityDataDF.hvplot.points("Lng", "Lat", s="Humidity", color = "City", geo = True, tiles = "OSM")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
cityCleanDF = cityDataDF.dropna(how = 'any')

# Display sample data
cityCleanDF.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,susuman,62.7833,148.1667,7.46,73,2,1.25,RU,1691690000
1,1,alo,7.4178,6.7371,25.84,90,97,1.51,NG,1691645201
2,2,ushuaia,-54.8000,-68.3000,4.81,81,75,4.12,AR,1691669126
3,3,saint-leu,-21.1500,55.2833,20.48,91,16,1.21,RE,1691635537
4,4,invercargill,-46.4000,168.3500,1.12,91,100,1.79,NZ,1691697271
5,5,nyborg,55.3127,10.7896,19.40,67,82,7.10,DK,1691638804
6,6,peace river,56.2501,-117.2860,16.17,94,75,2.57,CA,1691669352
7,7,port-aux-francais,-49.3500,70.2167,5.11,93,100,15.38,TF,1691634933
8,8,dabhol,17.6000,73.1667,24.99,88,7,1.30,IN,1691628523
9,9,coahuayana de hidalgo,18.7000,-103.6583,32.76,54,96,1.30,MX,1691670886


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotelDF = cityCleanDF[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotelDF["Hotel Name"] = ''

# Display sample data
hotelDF.head(15)

C:\Users\AzureDemise\AppData\Local\Temp\ipykernel_60700\677644233.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotelDF["Hotel Name"] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
0,susuman,RU,62.7833,148.1667,73,
1,alo,NG,7.4178,6.7371,90,
2,ushuaia,AR,-54.8000,-68.3000,81,
3,saint-leu,RE,-21.1500,55.2833,91,
4,invercargill,NZ,-46.4000,168.3500,91,
5,nyborg,DK,55.3127,10.7896,67,
6,peace river,CA,56.2501,-117.2860,94,
7,port-aux-francais,TF,-49.3500,70.2167,93,
8,dabhol,IN,17.6000,73.1667,88,
9,coahuayana de hidalgo,MX,18.7000,-103.6583,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel", "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotelDF.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotelDF.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotelDF.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotelDF.loc[index, 'City']} - nearest hotel: {hotelDF.loc[index, 'Hotel Name']}")

# Display sample data
hotelDF

Starting hotel search
susuman - nearest hotel: No hotel found
alo - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
saint-leu - nearest hotel: Iloha Seaview Hôtel
invercargill - nearest hotel: The Grand
nyborg - nearest hotel: Hotel Villa Gulle
peace river - nearest hotel: New Ridge Inn
port-aux-francais - nearest hotel: Keravel
dabhol - nearest hotel: Virgin Winds
coahuayana de hidalgo - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
bandarbeyla - nearest hotel: No hotel found
kongsvinger - nearest hotel: Vinger hotell
georgetown - nearest hotel: Page 63 hostel
tosontsengel - nearest hotel: Алтай
demyansk - nearest hotel: Гостиница РайПО
at taraf - nearest hotel: No hotel found
magnitogorsk - nearest hotel: Лагуна
al majaridah - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
adamstown - nearest hotel: No hotel found
revda - nearest hotel: 

leh - nearest hotel: Kang Lha Chen
fort st. john - nearest hotel: Fort St. John Motor Inn
san andres - nearest hotel: Hotel Decameron Isleño
korla - nearest hotel: Silver Star Hotel
talnakh - nearest hotel: Талнах
sitka - nearest hotel: Aspen Suites Hotel
lebu - nearest hotel: No hotel found
sanctuary point - nearest hotel: Dungowan Accommodation
ust'-ilimsk - nearest hotel: Усть-Илимск
happy valley-goose bay - nearest hotel: Hotel North Two
ola - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
martinho campos - nearest hotel: No hotel found
bodo - nearest hotel: Zefyr hotell
banes - nearest hotel: No hotel found
devils lake - nearest hotel: No hotel found
lapinlahti - nearest hotel: No hotel found
ar ruways - nearest hotel: No hotel found
mount pearl - nearest hotel: Sandman Signature St. John's
nikolayevsk-on-amure - nearest hotel: Север
nema - nearest hotel: No hotel found
halifax - nearest hotel: Four Points by Sheraton Halifax
porto novo - nearest hotel: Hô

arkhangel'sk - nearest hotel: Серафима
yuncheng - nearest hotel: 运城宾馆
kegen - nearest hotel: Kegen
biltine - nearest hotel: فندق ثلاث نجوم بلتن
chilliwack - nearest hotel: The Royal Hotel
sandakan - nearest hotel: Hotel City Star
grand falls-windsor - nearest hotel: Hotel Robin Hood
tarawa - nearest hotel: Betio Lodge
al burayqah - nearest hotel: فندق ماس ليبيا
bamboo flat - nearest hotel: Hotel Megapode Nest
golfito - nearest hotel: Bahia Marina Golfito Hotel
tsiombe - nearest hotel: No hotel found
hailar - nearest hotel: 呼伦贝尔融府商务酒店
la passe - nearest hotel: L'Estuaire
knyaze-volkonskoye - nearest hotel: No hotel found
sechura - nearest hotel: Hospedaje Claritza
al kharijah - nearest hotel: Kharga Hotel
port kembla - nearest hotel: Sage Hotel Wollongong
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
mwene-ditu - nearest hotel: Hôtel KAMAS
nizhniy odes - nearest hotel: No hotel found
aripuana - nearest hotel: No hotel found
juan griego - nearest hotel: Hotel Descanso Trop

palu - nearest hotel: Ujung Pandang Hotel
lensk - nearest hotel: No hotel found
kaduna - nearest hotel: Assa Pyramid Hotel
jayapura - nearest hotel: Hotel Jasmine Jayapura
ponta delgada - nearest hotel: Hotel Talisman
namtsy - nearest hotel: No hotel found
marinette - nearest hotel: Best Western Riverfront Inn
howrah - nearest hotel: Sun India Guest House
douentza - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,susuman,RU,62.7833,148.1667,73,No hotel found
1,alo,NG,7.4178,6.7371,90,No hotel found
2,ushuaia,AR,-54.8000,-68.3000,81,Apart Hotel Aires del Beagle
3,saint-leu,RE,-21.1500,55.2833,91,Iloha Seaview Hôtel
4,invercargill,NZ,-46.4000,168.3500,91,The Grand
...,...,...,...,...,...,...
586,ponta delgada,PT,37.7333,-25.6667,83,Hotel Talisman
587,namtsy,RU,62.7161,129.6658,67,No hotel found
588,marinette,US,45.1000,-87.6307,56,Best Western Riverfront Inn
589,howrah,IN,22.5892,88.3103,89,Sun India Guest House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotelDF.hvplot.points("Lng", "Lat", s="Humidity",hover_cols=['City', 'Country','Hotel Name','Lat','Lng','Humidity'], color = "City", geo = True, tiles = "OSM")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)